In [ ]:
# Import the pathlib, pandas, csv, and sys library
from pathlib import Path
import pandas as pd
import csv
import sys

# Set the file paths
menu_filepath = Path("./Resources/menu_data.csv")
sales_filepath = Path("./Resources/sales_data.csv")

# Initialize a lists
menu = []
sales = []
overperforming = []
underperforming = []

# @TODO: Initialize dict object to hold our key-value pairs of items and metrics
report = {}

# Initialize a row counter variable
row_count = 0

# Intialize variables
sum_total_count = 0
sum_total_revenue = 0
sum_total_cogs = 0
sum_total_profit = 0

# Define function to print stdout and write results to text file.
def print_to_both(input, write_to_file):
    ## Save a reference to the original standard output
    original_stdout = sys.stdout 
    ## print to original stdout
    print (input)
    ## change stdout to file and write
    sys.stdout = write_to_file
    print (input)
    ## change back to original stdout
    sys.stdout = original_stdout
       
# Open the csv file as an object
with open(menu_filepath, 'r') as csvfile:
    # Pass in the csv file to the csv.reader() function
    # (with ',' as the delmiter/separator) and return the csvreader object
    csvreader = csv.reader(csvfile, delimiter=",")
    # Read the header row first (skip this step if there is no header)
    header = next(csvreader)
    # Read each row of data after the header
    for row in csvreader:
        menu.append(row)
        
with open(sales_filepath, "r") as csvfile:
    # Pass in the csv file to the csv.reader() function
    # (with ',' as the delmiter/separator) and return the csvreader object
    csvreader = csv.reader(csvfile, delimiter=",") 
    # Read the header row first (skip this step if there is no header)
    header = next(csvreader)
    # Read each row of data after the header
    for row in csvreader:
        sales.append(row)
        
# @TODO: Loop over every row in the sales list object
for orders in sales:
    # Line_Item_ID,Date,Credit_Card_Number,Quantity,Menu_Item
    # @TODO: Initialize sales data variables
    quantity = int(orders[3])
    sales_item = orders[4]
    # If the item value not in the report, add it as a new entry with initialized metrics
    # Naming convention allows the keys to be ordered in logical fashion, count, revenue, cost, profit
    if not sales_item in report: 
        report[sales_item] = {}
        report[sales_item]['01-count'] = 0
        report[sales_item]['02-revenue'] = 0
        report[sales_item]['03-cogs'] = 0
        report[sales_item]['04-profit'] = 0 
    # @TODO: For every row in our sales data, loop over the menu records to determine a match
    for items in menu:
        # Item,Category,Description,Price,Cost
        # @TODO: Initialize menu data variables
        item = items[0]
        price = float(items[3])
        cost = float(items[4])
        # @TODO: Calculate profit of each item in the menu data
        profit = price - cost
        # @TODO: If the item value in our sales data is equal to the any of the items in the menu, then begin tracking metrics for that item
        if item == sales_item: 
            # @TODO: Print out matching menu data
            # @TODO: Cumulatively add up the metrics for each item key
            report[sales_item]["01-count"] += quantity
            report[sales_item]["02-revenue"] += price * quantity
            report[sales_item]["03-cogs"] += cost * quantity
            report[sales_item]["04-profit"] += profit * quantity
            break
        # @TODO: Else, the sales item does not equal any of the items in the menu data, therefore no match
        #elif not item in menu:
         #   print(f'{sales_item} does not equal {item}! NO MATCH!') 
    # @TODO: Increment the row counter by 1
    row_count += 1

# @TODO: Write out report to a text file (won't appear on the command line output)
# Set the header for report.csv
header = ['item', 'count', 'revenue', 'cogs', 'profit']

# Set the path for the report.csv
report_path = Path("report.csv")

# Open the output path as a file and pass into the 'csv.writer()' function
# Set the delimiter/separater as a ','
with open(report_path, "w") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=",")
    
    # Write the header as the first row
    csvwriter.writerow(header)
    
    # Loop over every key in analysis and write the associated key (name) and
    # nested key-value pairs (metrics)
    for row in report:
        csvwriter.writerow(
            [
                row,
                report[row]["01-count"],
                report[row]["02-revenue"],
                report[row]["03-cogs"],
                report[row]["04-profit"] 
            ])
        sum_total_count += report[row]["01-count"]
        sum_total_revenue += report[row]["02-revenue"]
        sum_total_cogs += report[row]["03-cogs"]
        sum_total_profit += report[row]["04-profit"]

# Create Dataframe for easier analyzing with pandas       
df = pd.DataFrame.from_dict(report, orient='index')

# Create list if conditions are met
overperforming = df[df["04-profit"] >= 60000].index.tolist()
underperforming = df[df["04-profit"] < 60000].index.tolist()


# Print results to orignal stdout and write to file.
with open('analysis.txt', 'w') as analysis_file:
    # @TODO: Print total number of records in sales data
    print_to_both (f'Total records processed: {row_count}\n', analysis_file)
    print_to_both ('Ramen Analysis Report', analysis_file)
    print_to_both ('----------------------------', analysis_file)
    print_to_both ('Summary Statistics:\n', analysis_file)
    print_to_both (f'Total Number of Ramen Sold: {sum_total_count:,} bowls', analysis_file)
    print_to_both (f'Total Revenue Generated: ${sum_total_revenue:,.2f}', analysis_file)
    print_to_both (f'Total Cost of Goods Sold: ${sum_total_cogs:,.2f}', analysis_file)
    print_to_both (f'Net Profit Generated: ${sum_total_profit:,.2f}\n', analysis_file)

    print_to_both ('Average Statistics:\n', analysis_file)
    print_to_both (f'Number of Ramen Types: {len(report)}', analysis_file)
    print_to_both (f'Average Number Sold per Ramen Type: {sum_total_count/len(report):,.2f} bowls', analysis_file)
    print_to_both (f'Average Revenue Generated per Ramen Type: ${sum_total_revenue/len(report):,.2f}', analysis_file)
    print_to_both (f'Average Cost of Golds Sold per Ramen Type: ${sum_total_cogs/len(report):,.2f}', analysis_file)
    print_to_both (f'Average Net Profit Generated per Ramen Type: ${sum_total_profit/len(report):,.2f}\n', analysis_file)

    print_to_both ('Min/Max Statistics:\n', analysis_file)
    print_to_both (f'Most Popular: ({df["01-count"].max()}, {df["01-count"].idxmax()})', analysis_file)
    print_to_both (f'Least Popular: ({df["01-count"].min()}, {df["01-count"].idxmin()})', analysis_file)
    print_to_both (f'Most Revenue Generating: ({df["02-revenue"].max()}, {df["02-revenue"].idxmax()})', analysis_file)
    print_to_both (f'Least Revenue Generating: ({df["02-revenue"].min()}, {df["02-revenue"].idxmin()})', analysis_file)
    print_to_both (f'Most Costly: ({df["03-cogs"].max()}, {df["03-cogs"].idxmax()})', analysis_file)
    print_to_both (f'Least Costly: ({df["03-cogs"].min()}, {df["03-cogs"].idxmin()})', analysis_file)
    print_to_both (f'Most Profitable: ({df["04-profit"].max()}, {df["04-profit"].idxmax()})', analysis_file)
    print_to_both (f'Least Profitable: ({df["04-profit"].min()}, {df["04-profit"].idxmin()})\n', analysis_file)

    print_to_both ('Filter Items:\n', analysis_file)
    print_to_both (f'Overperforming Items: {overperforming}', analysis_file)
    print_to_both (f'Underperforming Items: {underperforming}', analysis_file)